# HyP3 prototype water extent map product

This notebook mimics the workflow a HyP3 Water Map plugin will follow to create an On Demand surface water extent map. It is intended demonstrate *how* HyP3 goes from an On Demand request to a product as well as provide a platform for testing/tuning/refinement ot the water extent map product.

Importantly, step 2 will become the plugin's primary interface, and the options provided to the `make_water_map` function are able to be exposed to users. Step 1 is a required data marshaling step that will be handled transparently inside the container.

If you'd like an overview of the water extent mapping process, run the following cell to get the `make_water_map` documentation, otherwise, skip to step 1.

In [ ]:
from asf_tools.water_map import make_water_map

make_water_map?

## 1. Generate input dual-pol RTC product

Water extent mapping requires Radiometrically Terrain Corrected (RTC) Sentinel-1 scenes in order to accurately connect backscatter features to the input HAND data. The next few cells will use HyP3's Python SDK to request an RTC product from HyP3 for the Sentinel-1 granule (scene):

[S1A_IW_GRDH_1SDV_20210413T235641_20210413T235706_037439_0469D0_3F2B](https://search.asf.alaska.edu/#/?zoom=6.964&center=-90.509,40.296&resultsLoaded=true&granule=S1A_IW_GRDH_1SDV_20210413T235641_20210413T235706_037439_0469D0_3F2B-METADATA_GRD_HD&searchType=List%20Search&searchList=S1A_IW_GRDH_1SDV_20210413T235641_20210413T235706_037439_0469D0_3F2B)

### 1.1 Connect to HyP3

In [ ]:
import logging
import shutil
import hyp3_sdk as sdk

logging.basicConfig(level=logging.INFO)

hyp3 = sdk.HyP3(prompt=True)

### 1.2  Submit an On Demand request for the input RTC product

Currently, the RTC options provided to `hyp3.submit_rtc_job` as specified are expected/required by `make_water_map` and are not recommended to be changed, except for:

* `speckle_filter` -- Apply an Enhanced Lee speckle filter.

* `dem_matching` -- Coregisters SAR data to the DEM, rather than using dead reckoning based on orbit files. See the [Terrain Correction](https://hyp3-docs.asf.alaska.edu/guides/rtc_product_guide/#terrain-correction) in the RTC Product Guide for more information.

* `include_dem` -- include a GeoTIF of the Copernicus DEM pixel aligned to the RTC products. This GeoTIFF isn't used by `make_water_map` but was used to create the RTC product and the HAND data

* `include_rgb` -- include a false-color RGB decomposition for dual-pol granules. This GeoTIFF isn't used by `make_water_map` but may help you [interpret the RTC image](https://github.com/ASFHyP3/hyp3-lib/blob/develop/docs/rgb_decomposition.md#interpreting-the-images) and evaluate the water extent map


In [ ]:
granule = 'S1A_IW_GRDH_1SDV_20210413T235641_20210413T235706_037439_0469D0_3F2B'

job = hyp3.submit_rtc_job(
    granule, name='water-extent-example',
    radiometry='gamma0', scale='power', resolution=30, dem_name='copernicus',
    include_dem=True, include_rgb=True,
    speckle_filter=True, dem_matching=False
)
print(job)

### 1.3 Wait for the RTC product to be generated

This will take $\approx$ 25 minutes to complete.

In [ ]:
job = hyp3.watch(job)

### 1.4 Download the RTC product

RTC products are downloaded as Zip files and then extracted. The GeoTIFF for the VV and VH polarizations are selected from the (unzipped) product folder.

In [ ]:
product_zip = job.download_files()[0]
shutil.unpack_archive(product_zip)
vv_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VV.tif'
vh_raster = product_zip.parent / product_zip.stem / f'{product_zip.stem}_VH.tif'
print(f'VV raster: {vv_raster}')
print(f'VH raster: {vv_raster}')

## 2. Generate the water extent map

Now that we have marshalled all the required input data, we can generate the surface water extent map!

**Note:** this will take $\approx$ 30 minutes to complete.

In [ ]:
from asf_tools.water_map import make_water_map

water_extent_raster = 'water_extent.tif'
make_water_map(water_extent_raster, vv_raster, vh_raster, tile_shape=(100, 100),
               max_vv_threshold=-15.5, max_vh_threshold=-23., hand_threshold=15., hand_fraction=0.8)

`make_water_map` in [water_map.py](../asf_tools/water_map.py) is the primary "science" interface for generating the water maps. It *should* include all the necessary options for tuning the product -- if it doesn't, and you'd like other things exposed, either:
* Ask us to expose it by:
  * [Chatting with us in Gitter](https://gitter.im/ASFHyP3/community)
  * [Opening an issue](https://github.com/ASFHyP3/asf-tools/issues/new/choose)

* [Fork](https://github.com/ASFHyP3/hyp3-sdk/fork) and propose changes in a Pull Request. See our [contributing guidelines](https://hyp3-docs.asf.alaska.edu/contributing/).

For more info on the `make_water_map` function, run the cell below.

In [ ]:
from asf_tools.water_map import make_water_map
make_water_map?